In [1]:
from refinery import Client

# if you run the application locally, use "http://localhost:4455" as the uri
client = Client.from_secrets_file("secrets.json")

df = client.get_record_export(tokenize=False)

ℹ Connecting to https://app.dev.kern.ai
✔ Logged in to system.


In [2]:
filtered_df = df[
    (df["__Topic__WEAK_SUPERVISION__confidence"].astype(float) > 0.7) # either threshold WS label
    | ~(df["__Topic__MANUAL"].isnull()) # or manual label
    ]

test_idx = [not elem for elem in df.index.isin(filtered_df.index)]
test_df = df.loc[test_idx][["Title", "Description", "Class Index"]]

In [21]:
translation_dict = {
            "World" : 1,
            "Sports" : 2,
            "Business" : 3,
            "Sci/Tech" : 4
        }

translat = ["", "World", "Sports", "Business", "Sci/Tech"]

test_df["label"] = test_df["Class Index"].apply(lambda x: translat[int(x)])

In [3]:
def combine_labels(row):
    if(row["__Topic__MANUAL"] is not None):
        return row["__Topic__MANUAL"]
    else:
        return row["__Topic__WEAK_SUPERVISION"]

filtered_df["label"] = filtered_df.apply(combine_labels, axis=1)

In [7]:
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(filtered_df.index, test_size=0.25, random_state=42)

filtered_df_val = filtered_df[["Title", "Description", "label"]].loc[val_idx]
filtered_df_train = filtered_df[["Title", "Description", "label"]].loc[train_idx]

test_df[["Title", "Description", "label"]].to_json("labeled_data_test.json", orient="records")
filtered_df_val[["Title", "Description", "label"]].to_json("labeled_data_val.json", orient="records")
filtered_df_train[["Title", "Description", "label"]].to_json("labeled_data_train.json", orient="records")